## Question 5
Solve DSGE model numerically

\begin{split}
u(c_t) & = \frac{c^{1-\gamma}_t -1}{1-\gamma}      \\
F(K_t,L_t,z_t) & = K^{\alpha}_t (L_te^{z_t})^{1-\alpha}  \nonumber
\end{split}


\begin{align*}
c_t &= (1-\tau)(w_t+ (r_t -\delta)k_t) + k_t + T_t - k_{t+1} \\
\frac{1}{c_t^{-\gamma}} &= \beta E_t \big\{ \frac{1}{c_{t+1}^{-\gamma}}((r_{t+1} - \delta)(1-\tau)+ 1) \big\} \\
r_t &= \alpha (e^{zt})^{(1-\alpha)} K_t^{\alpha-1} \\
w_t &= (1-\alpha) (e^{zt})^{(1-\alpha)} K_t^{\alpha} \\
\tau(w_t + (r_t -\delta)k_t) &= T_t \\
z_t &= (1-p_z)\bar{z} + p_zz_{t-1} + \epsilon^z_t,  \epsilon^z_t \sim \text{i.i.d}(0, \sigma^2_z)
\end{align*}

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from collections import namedtuple

In [10]:
params = ["gamma", "beta", "alpha", "delta", "zbar", "tau", "sigma", "rho", "xi", "a"]
DSGE = namedtuple("BaselineModel", params)

q5 = DSGE(2.5, 0.98, 0.40, 0.10, 0.0, 0.05, 0.02, 0.95, 1.5, 0.05)

In [82]:
def get_jumps(k, l=1, params=q5):
    '''
    Given a level of capital/labor and a named tuple of parameters, this function will return 
    all the jump variables. 
    
    Inputs are:
        k: capital
        l: labor
        z: random shock to productivity
        params: named_tuple of parameter values
    
    Output are:
        Y: output
        w: wage rate
        r: rental rate on capital
        c: consumption
        i: investment
    '''
    w = (1 - params.alpha)*(k**params.alpha)*(l**(-params.alpha))
    r = params.alpha * ((k)**(params.alpha-1)) * (l ** (1 - params.alpha))
    y = (k ** params.alpha)*((l * np.exp(params.zbar)) ** (1 - params.alpha))

    c = w * l + (r - params.delta) * k
    u = (c ** (1 - params.gamma)) - 1 / (1 - params.gamma)
    
    i = params.delta * y
    
    return w,r,y,c,u, i

def inter_ee(k, *args):
    '''
    For a given level of capital, 
    
    Inputs are:
        k: capital
        args: various parameters of the model
    
    Output are:
        Error: optimal allocation is when error=0
    '''
    gamma, beta, tau, delta = args # params
    
    w,r,y,c,u,i = get_jumps(k=k) # get jump vars
    
    error = (c ** -gamma) - beta * (c ** -gamma) * ((1 - tau) * (r - delta) + 1)
    return error

In [94]:
params = q5
args = (params.gamma, params.beta, params.tau, params.delta)

ss_guess = 1
kbar = opt.root(inter_ee, ss_guess, args=args).x # root of euler is steady state
w,r,y,c,u,i = get_jumps(k=kbar)

print("QUESTION 5")
print("Capital steady state = ", kbar)
print("Output steady state = ", y)
print("Investment steady state = ", i)

QUESTION 5
Capital steady state =  [7.28749795]
Output steady state =  [2.21325461]
Investment steady state =  [0.22132546]


## Question 6

In [100]:
# Similar to above but now two euler equations, labor/leisure and intertemporal
def interintra_ee(state, *args):
    gamma, alpha, beta, delta, tau, a, xi = args
    k, l = state
    w, r, y, c, u, i = get_jumps(k=k, l=l) # get jump vars
    
    error1 = (c**(-gamma)) - beta*(c**(-gamma))*((1 - tau)*(r - delta) + 1)
    error2 = a*((1 - l) ** (-xi))- w * (c**(-gamma)) *(1 - tau)
    return np.array([error1, error2])

In [106]:
q6 = DSGE(2.5, 0.98, 0.40, 0.10, 0.0, 0.05, 0.02, 0.95, 1.5, 0.5)
args = (q6.gamma, q6.alpha, q6.beta, q6.delta, q6.tau, q6.a, q6.xi)

ss_guess = np.array([7,0.3])
k, l = opt.root(interintra_ee, ss_guess, args=args).x
w, r, y, c, u, i = get_jumps(k=k, l=l)

print("Steady State Capital:", k)
print("Steady State labor:", l)
print("Steady State Output:", y)
print("Steady State Investment:", i)

Steady State Capital: 4.225229026798704
Steady State labor: 0.5797914531667254
Steady State Output: 1.2832261088340167
Steady State Investment: 0.12832261088340166
